In [1]:
!pip install selenium
!pip install webdriver-manager
!pip install bs4
!pip install csv
!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [2]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys

# Initialize the Chrome driver
# service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome()

# Navigate to the website
url = 'https://m.dailyhunt.in/news/india/odia/for+you?mode=pwa&langchange=true&launch=true'
driver.get(url)
time.sleep(5)

def scroll_to_bottom(driver):
    # Get the current scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for the page to load
        # time.sleep(2)

        # Calculate the new scroll height and compare with the last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If the heights are the same, we've reached the bottom
            break
        last_height = new_height

# Initialize the lists to store buttons and links
read_more_buttons = []
read_more_links = []

i = 1
# Keep finding "Read more" buttons until we have 32 of them
while len(read_more_links) < 3000:

    try:
        xpath = f'/html/body/div[2]/div/div/div/section[1]/div/div/div/div[{i}]/section/section/div/section/a'
        new_button = driver.find_element(By.XPATH, xpath)

        if new_button.get_attribute('href') not in read_more_links:
            read_more_links.append(new_button.get_attribute('href'))

        print(len(read_more_links))

        i += 1
        scroll_to_bottom(driver)
        # time.sleep(1)
    except:
        pass


# for rea in read_more_links:
#     print(rea)
#/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/div/section/a
#/html/body/div[2]/div/div/div/section[1]/div/div/div/div[2]/section/section/div/section/a
#/html/body/div[2]/div/div/div/section[1]/div/div/div/div[3]/section/section/div/section/a
#/html/body/div[2]/div/div/div/section[1]/div/div/div/div[4]/section/section/div/section/a
#/html/body/div[2]/div/div/div/section[1]/div/div/div/div[5]/section/section/div/section/a
#/html/body/div[2]/div/div/div/section[1]/div/div/div/div[6]/section/section/div/section/a

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/126.0.6478.63/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x59db3067069a <unknown>
#1 0x59db303530dc <unknown>
#2 0x59db3038802a <unknown>
#3 0x59db3038443b <unknown>
#4 0x59db303ceef9 <unknown>
#5 0x59db303c2613 <unknown>
#6 0x59db303924f7 <unknown>
#7 0x59db30392e4e <unknown>
#8 0x59db3063686b <unknown>
#9 0x59db3063a911 <unknown>
#10 0x59db3062235e <unknown>
#11 0x59db3063b472 <unknown>
#12 0x59db30606cbf <unknown>
#13 0x59db30660098 <unknown>
#14 0x59db30660270 <unknown>
#15 0x59db3066f7cc <unknown>
#16 0x7926b9010ac3 <unknown>


In [ ]:
# Remove the headlines and continue with the rest of your code
# Create a CSV file and write the header row
csv_file = open('news_data_odia.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Headline', 'URL', 'Image URL', 'Content', 'Comments'])

def extract_content(link):
    try:
        driver.get(link)
        time.sleep(1)

        # Find comments
        comments = driver.find_elements(By.XPATH, '//ul//li//div[@class="comment_para"]/p')

        # If there are comments, return True, else False
        if comments:
            driver.back()
            time.sleep(1)
            return True
        else:
            driver.back()
            time.sleep(1)
            return False
    except:
        pass

def print_comments(link):

    try:
        driver.get(link)
        time.sleep(1)

        # Find headline
        headline_tag = driver.find_elements(By.XPATH, './/h2')
        headline = headline_tag[1].text
        print("HEADLINE: ", headline)

        # Find comments again
        comments = driver.find_elements(By.XPATH, '//ul//li//div[@class="comment_para"]/p')

        # Find images with alt text containing the headline
        images = driver.find_elements(By.TAG_NAME, 'img')
        image_urls = []
        for image in images:
            alt_text = image.get_attribute('alt')
            if headline.lower() in alt_text.lower():
                image_urls.append(image.get_attribute('src'))
                print("IMAGES: ", image.get_attribute('src'))

        # Find the div with class "disclaimer"
        disclaimer_div = driver.find_element(By.XPATH, '//div[@class="disclaimer"]')

        # Extract the HTML source before the "disclaimer" div
        html_source = driver.page_source
        disclaimer_div_start = html_source.find(disclaimer_div.get_attribute('outerHTML'))
        relevant_html = html_source[:disclaimer_div_start]

        # Create a new instance of BeautifulSoup with the relevant HTML
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(relevant_html, 'html.parser')

        # Find and get the paragraph content
        paragraphs = soup.find_all('p')
        content = '\n'.join([paragraph.get_text() for paragraph in paragraphs])
        print("\nCONTENT:\n")
        print(content)

        # Get the comments text
        comments_text = '\n'.join([f"{i}. {comment.text}" for i, comment in enumerate(comments, start=1)])

        print("\nCOMMENTS:\n")
        print(comments_text)

        print("\n" + "-" * 100 + "\n")
        driver.back()
        time.sleep(1)

        return image_urls, content, comments_text, headline
    except:
        pass

j = 1
for link in read_more_links:
    try:
        print(j)
        j += 1
        if extract_content(link):
            print("LINK: ", link)
            image_urls, content, comments_text, headline = print_comments(link)
            csv_writer.writerow([headline, link, ', '.join(image_urls), content, comments_text])
    except:
        continue
# Close the CSV file
csv_file.close()
driver.quit()